In [1]:
import numpy as np
import pandas as pd
import jieba
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [2]:
import pickle

In [3]:
with open('./news_content_words.pkl', 'rb') as f:
    news_content_words = pickle.load(f)

In [4]:
news_content_words[0]

'原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 中 裸身 奔走 \r\n 深圳 交警 微博称 昨日 清晨 交警 发现 女子 赤裸 上身 行走 南坪 快速 期间 轻生 年头 一辅警 发现 赶紧 披上 黄衣 一路 劝说 \r\n 事发 时 \r\n 到底 发生 \r\n 南 记者 带 一起 还原 现场 \r\n 南 记者 龙岗 大队 坂田 中队 见到 辅警 刘青 发现 女生 辅警 一位 外表 高大 帅气 说话 略带 腼腆 90 青年 \r\n 刘青 介绍 月 16 日 早上 时 36 分 正在 环城 南路 附近 值勤 接到 中队 一位 女子 裸身 进入 机动车 可能 危险 警情 骑着 铁骑 沿路 寻找 大概 花 十多分钟 南坪 大道 坂田 出口 龙岗 方向 逆行 辅道 发现 女子 \r\n 女子 身上 一丝不挂 地逆 车流 而行 时 走时 停 时 坐 时 躺 险象环生 刘青停 铁骑 一名 巡防 员 追 上去 发现 女子 情绪 低落 话 刘青 尝试 女子 交流 劝说 女子 离开 女子 愿意 接受 继续 缓慢 地往 南坪 快速路 主干道 走 \r\n 路边 已经 聚集 市民 围观 刺激 女子 情绪 刘青 巡防 员 一边 盯 女子 一边 驱赶 围观 群众 \r\n 现场 还原 \r\n 警方 提供 一份 视频 了解 16 日 早上 时 25 分 女子 出现 坂雪岗 大道 环城 南路 监控 视频 中 女子 穿着 白色 内裤 正 坂雪岗 大道 南坪 快速 方向 缓慢 走 \r\n 当时 上班 高峰期 十字路口 车流 已经 排起 长队 女子 出现 时 路上 市民 纷纷 驻足 观望 不少 车辆 放慢 速度 女子 市民 观望 停下 脚步 依然 缓慢 走 女子 行进 十字路口 中间 时 一辆 货车 挡住 镜头 女子 再次出现 镜头 时 发现 女子 已经 没 穿 内裤 全身 裸露 继续 南坪 快速 方向 走 记者 发现 视频 中 女子 周围 没有 尾随 上前 劝止 市民 \r\n 大清早 路上 看到 情况 \r\n 恐怕 没 办法 淡定 \r\n 面对 这一 情况 \r\n 刘青 表示 根本 不敢 心里 挺 别扭 感觉 尴尬 刘青 跟随 女子 南坪 快速路 主干道 时 女子 作出 意想不到 举动 突然 靠近 护栏 上面 跳下去 刘青 赶忙 冲上去 拉住 女子 手 控制 住 远离

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
vectorizer = TfidfVectorizer(max_features=10000)

In [8]:
X = vectorizer.fit_transform(news_content_words)

In [9]:
X.shape

(76925, 10000)

In [10]:
with open('news_source.pkl', 'rb') as f:
    y = pickle.load(f)

In [11]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [24]:
len(y)

76925

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [26]:
len(y_train)

53847

In [13]:
lr_clf = LogisticRegression()

In [14]:
param_grid = [{'C': [i for i in range(1,6)]}]

In [15]:
grid_search = GridSearchCV(lr_clf, param_grid, scoring='accuracy', n_jobs=-1, verbose=3)

In [16]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/anaconda3/envs/Kstone/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    6.3s finished
/anaconda3/envs/Kstone/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'C': [1, 2, 3, 4, 5]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=3)

In [17]:
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train,cv=3,scoring='accuracy')
print (scores.mean(), scores)

/anaconda3/envs/Kstone/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9868702062984669 [0.9870195  0.98657307 0.98701805]


In [19]:
lr = grid_search.best_estimator_

In [20]:
lr

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [22]:
lr.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1])

In [23]:
scores = cross_val_score(grid_search.best_estimator_, X_test, y_test,cv=3,scoring='accuracy')
print(scores.mean(), scores)

/anaconda3/envs/Kstone/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9798509120485276 [0.97933186 0.98102171 0.97919917]
